# Элементы профессионального программирования

Алексей Умнов https://www.youtube.com/watch?v=n6PgUI-TUMU  
Слайды доступны по адресу: http://parallels.nsu.ru/~fat/Python/

## Итераторы. Введение

Зачем нужны интераторы?

````python
for i in range(10 ** 6): # от 0 до 1 000 000
    ...
````

* В памяти будет хранится список из миллиона чисел которые нагенерит функция `range()` и это плохо
* Можно обойтись двумя числами:
  * Неудобный способ:
  ````python
  end = 10 ** 6
  i = 0
  while i < end:
    ...
    i += 1
  ````
  * Правильный способ, использовать xrange() (для Python v2.x):
  ````python
  for i in xrange(10 ** 6):
    ...
  ````

| Python 2 | Python 3 |
|----------|----------|
| range    | ---      |
| xrange   | range    |

Далее рассмотрим:
* Как работает функция `xrange()` и как она взаимодействует с циклом `for`?
* Где можно использовать итераторы?
* Как создавать итераторы?

## Функции-Генераторы

Наличие ключевого слова `yield` внутри функции говорит о том, что данная функция является не обычной функцией, а функцией генераторов. Функция превращается в такой объект, у которого результат что-то вроде списка, при чём `yield` здесь является некоторым аналогом `return`, оно вроде как возвращает очередное значение, но не совсем возвращает, а так сказать добавляет в список элементы которые нужно вернуть из функции.

Если использован оператор `yield`, то уже нельзя использовать обычный оператор `return`, потому что Python функцию сильно изменяет и она теперь возвращает не конкретное какое-то одно значение, а объект, который по очереди передает вот эти три значения, например, которые вы передали в `yield`:

In [1]:
def g():
    yield 1
    yield 'abc'
    yield [1, 2, 3]

for x in g(): # Будет 3 итерации по функции 'g'
    print(x)

1
abc
[1, 2, 3]


Пример генератора `range`:

In [2]:
def myrange(begin, end, step):
    while begin < end:
        yield begin      # Итератор. При каждом обращении к функции myrange() вернет следующее значение
        begin += step

x = myrange(0, 10, 2) # Функция не запускается, а создается объект типа generator
print(x)              # <generator object myrange at 0x7fe7cbb9c048>
print(list(x))        # [0, 2, 4, 6, 8]

<generator object myrange at 0x7f20dcadfa40>
[0, 2, 4, 6, 8]


In [4]:
x = myrange(0, 10, 2) # Функция не запускается, а создается объект типа generator
for i in x:
    print(i)

0
2
4
6
8


### Схема работы генераторов

Вызов функции-генератора:
* Позиция = начало функции

Необходимость отдать очередной элемент:
* Начать исполнение функции с текущей позиции
* Если встретился оператор yield
 * Отдать аргумент оператора yield
 * Остановить исполнение
 * Запомнить текущую позицию
* Иначе сообщить, что больше элементов нет

### Генератор random_length_repeat()

Пример генерации случайного количества размножения объектов:

In [11]:
import random

def random_length_repeat(obj, i): # Итерируемая функция-генератор
    while True:
        if random.random() < i:
            break
        yield obj

list(random_length_repeat('a', 0.2)) # Создается переменое число элементов

['a', 'a', 'a', 'a', 'a', 'a']

In [18]:
list(random_length_repeat('a', 0.2))

['a', 'a', 'a', 'a', 'a', 'a']

In [20]:
[x for x in random_length_repeat('a', 0.2)]

['a', 'a', 'a', 'a']

In [16]:
[x * 2 for x in random_length_repeat('a', 0.2)]

['aa', 'aa', 'aa', 'aa', 'aa', 'aa', 'aa', 'aa', 'aa', 'aa', 'aa', 'aa']

### Списковое выражение

In [21]:
[x for x in range(5)]

[0, 1, 2, 3, 4]

### Генераторное выражение

Передается как генератор. Можно передавать в другую функцию как аргумент, вместо готового списка и итерировать с помощью цикла `for`:

In [22]:
(x for x in range(5))

<generator object <genexpr> at 0x7f20dcadfa40>

### Упрощенное генераторное выражение

Передается как генератор. Круглые скобки не обязательны, это эквивалентно предыдущему примеру:

In [24]:
print(x for x in range(5))

<generator object <genexpr> at 0x7f20dcadffc0>


## Итераторы. Теория

Последвательность (sequence) - упорядочненный набор объектов, к элементам которого можно обращаться по индексу.

Должны быть определены операции:
* `__len__`
* `__getitem__` (для индексов)

Примеры последовательностей:
* list
* tuple
* str

Цикл `for` для последовательностей. Что имитируем:

  ````python
   for x in sequence:
     action(x)
  ````

Имитация цикла `for` с последоательностью:

````python
def for_sequence(sequence, action):
    length = len(sequence)
    i = 0
    while i < length:
        x = sequence[i]
        action(x)
        i += 1
````

## Итерируемые и итераторы

Итерируемое (iterable) - упорядочненный набор объектов, элементы которого можно получать по одному. Должна быть определена операция `__iter__` (возвращает итератор).

Итератор (iterator) - объект, представляющий поток данных. Должна быть определена операция `__next__` (возвращает очередной объект). Окончание потока: исключение `StopIteration`.

Цикл `for` для **iterable**. Что имитируем:

````python
for x in iterable:
    action(x)
````

Имитация цикла `for` с итерируемым объектом:

````python
def for_iterable(iterable, action):
    iterator = iter(iterable) # Встроенная функция iter обращается к методу объекта __iter__
    try:
        while True:
            x = next(iterator) # Встроенная функция next обращается к методу объекта __next__
            action(x)
    exception StopIteration:
        pass
````

Пример итератора и интерируемого объекта:

### Код для Iterator (Итератор)

In [25]:
class MyRangeIterator():

    def __init__(self, end):
        self.end = end
        self.current = 0

    def __next__(self):
        if self.current == self.end:
            # Исключение можно кидать, а можно не кидать, чтобы сделать бесконечный итератор-генератор:
            raise StopIteration()
        result = self.current
        self.current += 1
        return result

### Код для Iterable (Итерируемое)

In [26]:
class MyRange():

    def __init__(self, end):
        self.end = end

    def __iter__(self):
        # А можно вернуть здесь self и в этом классе реализовать метод __next__,
        # тогда будет Итерируемое + Итератор в одном
        return MyRangeIterator(self.end)

### Использование итератора для итерируемого

In [28]:
x = MyRange(5)

for i in x:
    print(i)

0
1
2
3
4


Каждый вызов, `list` видит, что это итерируемое, вызывает у него функцию `__iter__`, чтобы получить итератор
и каким-то внутренним циклом `for` по итератору проходится и все элементы из него извлекаются. Два вызова `list(x)` они совершенно независимы, каждый из них создает свой итератор от 0 до 4:

In [31]:
list(x)

[0, 1, 2, 3, 4]

In [32]:
list(x)

[0, 1, 2, 3, 4]

А генераторы ведут себя не так. Вызовом `myrange(0, 5, 1)` создается новый объект типа `generator` и присваивается переменной `y`:

In [37]:
def myrange(begin, end, step):
    while begin < end:
        yield begin      # Итератор. При каждом обращении к функции myrange() вернет следующее значение
        begin += step


y = myrange(0, 5, 1)

list(y)

[0, 1, 2, 3, 4]

Возвращает пустой список. Истощение генератора. Генератор проходится по элементам только 1 раз:

In [38]:
list(y)

[]

Однако, если вызвать генератор не через переменную, т.е., каждый раз создавая новый объект, то данные будут возвращаться. Создается новый объект типа `generator`, поэтому генератор не истощается:

In [39]:
list(myrange(0, 5, 1))

[0, 1, 2, 3, 4]

In [40]:
list(myrange(0, 5, 1))

[0, 1, 2, 3, 4]

Итерируемые и истощение:

* Последовательность - list, tuple, str, ...
  * Итерируемое
  * Не истощается

* Итерируемое, но не последовательность может:
  * Истощаться (генераторы)
  * Не истощаться (xrange или range в Python 3)

* Итератор
  * Итерируемое
  * Истощается

Итерируемость итераторов:

`x` - итератор
  * `iter(x)` - возвращает `x`
  * `x` истощается

Пример использования в функции. Правда ли, что все элементы в нем равны? :)  
Не самый простой способ, можно сделать по другому, но мне хочется так:

````python
def all_equal(iterable):
    iterator = iter(iterable)
    first_element = next(iterator)
    for element in iterator:
        if element != first_element:
            return False
        return True
````

В примере есть логическая ошибка. Что будет, если `iterable` пустой или с одним элементом?

## Инструменты функционального программирования

### Функция map()

`map(func, iterable)`
* Применяет `func` ко всем элементам `iterable`
* Возвращает список результатов

In [41]:
x = map(str, [1, 2, 3]) # Применяет ко всем элементам списка функцию str()

list(x)

['1', '2', '3']

`map(lambda, списковое выражение)`:

In [42]:
x = map(lambda x: x + 1, [1, 2, 3])

list(x)

[2, 3, 4]

Но лучше использовать аналогичную запись, вместо функции `map()`, чтобы было меньше ключевых слов (`map`, `lambda`, `list`, ...), лучше было читать:

In [43]:
x = [x + 1 for x in [1, 2, 3]]

print(x)

[2, 3, 4]


Еще примеры для `map()`:

Пользователь вводит числа: `1 2 3 4 5`:

In [45]:
numbers = map(int, input("Введите несколько чисел через пробел и нажмите <Enter>: ").split())

print('Результат:', list(numbers)) # Все введенные пользователем числа преобразованы к типу int

Введите несколько чисел через пробел и нажмите <Enter>: 1 2 3 4 5
Результат: [1, 2, 3, 4, 5]


Аналогично можно было бы использовать списковый генератор:

In [46]:
x = [int(x) for x in input("Введите несколько чисел через пробел и нажмите <Enter>: ").split()]

print(x)

Введите несколько чисел через пробел и нажмите <Enter>: 1 2 3 4 5
[1, 2, 3, 4, 5]


Подсчет длин слов в тексте:

In [47]:
text = 'Dog barks cat meows'

x = map(len, text.split())

print(x)
print(list(x))

[3, 5, 3, 5]


`map` - итерируемое, можно использовать цикл:

In [49]:
x = map(len, text.split())

for i in x:
    print(i)

3
5
3
5


### Функция enumerate()

enumerate(iterable)
* Нумерует элементы в iterable
* Возвращает истощаемый iterable

In [50]:
for i, e in enumerate(['a', 'b', 'c']):
    print(i, e)

0 a
1 b
2 c


### Функция filter()

filter(func, iterable)
* Вычисляет значение func для всех элементов iterable
* Возвращает список элементов, для которых функция вернула True
* Как и функция map возвращает итератор (для Python 3)

Функция callable() может сказать, является объект функцией или нет (есть ли у него операция "круглые скобки"):

In [2]:
# Только 2 объекта являются функциями, list и tuple не являются функциями:
x = filter(callable, [[], int, (), abs])

list(x)

[int, <function abs>]

In [3]:
x = filter(lambda x: x%2 == 0, [1, 2, 3, 4, 5, 6])

list(x)

[2, 4, 6]

Аналогичная и лучше читаемая запись (если нужно получить весь список сразу):

In [4]:
x = [x for x in [1, 2, 3, 4, 5, 6] if x%2 == 0]

print(x)

[2, 4, 6]


### Функция reduce()

Эта функция устарела для Python 3, лучше использовать цикл `for` для этого:

````python
x = reduce(lambda x,y: x + y, [1, 2, 3, 4, 5])
````

## Библиотека itertools (for Python 2)

Итераторные аналоги. Возвращают не список, а итератор.  
Эти функции для Python 2 !!! В Python 3 функции map, zip, filter возвращают итератор!

````python
imap(func, iterable)
  * Итераторный аналог map
  ````
  
````python
izip(p, q, ...)
  * Итераторный аналог zip
````

````python
ifilter(func, iterable)
  * Итераторный аналог filter
````